In [4]:
from scipy.stats import binom
import numpy as np
import cvxpy as cp

In [5]:
import mosek

$$binom.pmf(k, n, p) = \binom{n}{k}p^k(1-p)^{n-k} $$

$$\tilde{r}|r \sim B(m, \frac{r - 1}{n-1}) + 1$$

$$ p = \frac{r-1}{n-1}$$

$$p(\tilde{r}|r) = \binom{m}{\tilde{r} - 1}p^{\tilde{r} - 1}(1-p)^{m + 1- \tilde{r}}$$

In [2]:
# rt ~ [1, m + 1]
# rt - 1 ~ [0, m]
# r ~ [1, n]

In [30]:
# Example:
m = 100
n = 10000
pr = 1/n # uniform prior

In [31]:
# rt is actually rt - 1 here
rt = np.tile(np.arange(m + 1), (n, 1))
rt

array([[  0,   1,   2, ...,  98,  99, 100],
       [  0,   1,   2, ...,  98,  99, 100],
       [  0,   1,   2, ...,  98,  99, 100],
       ...,
       [  0,   1,   2, ...,  98,  99, 100],
       [  0,   1,   2, ...,  98,  99, 100],
       [  0,   1,   2, ...,  98,  99, 100]])

In [32]:
r = np.tile(np.arange(1, n + 1).reshape(-1, 1), (1, m + 1))
r

array([[    1,     1,     1, ...,     1,     1,     1],
       [    2,     2,     2, ...,     2,     2,     2],
       [    3,     3,     3, ...,     3,     3,     3],
       ...,
       [ 9998,  9998,  9998, ...,  9998,  9998,  9998],
       [ 9999,  9999,  9999, ...,  9999,  9999,  9999],
       [10000, 10000, 10000, ..., 10000, 10000, 10000]])

In [33]:
p = (r - 1.)/( n - 1)

In [34]:
prr = binom.pmf(rt, m, p)

In [43]:
# Considering M(r) = Recall@10
Mr = np.zeros(n)
Mr[:20] = 1

In [44]:
F = np.zeros((m, m + 1))
for i in range(m):
    F[i][i] = 1
    F[i][i+1]=-1
F

array([[ 1., -1.,  0., ...,  0.,  0.,  0.],
       [ 0.,  1., -1., ...,  0.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ..., -1.,  0.,  0.],
       [ 0.,  0.,  0., ...,  1., -1.,  0.],
       [ 0.,  0.,  0., ...,  0.,  1., -1.]])

$$F = \begin{bmatrix}
1&-1&0&\cdots&0\\
0&1&-1&\cdots&0\\
\vdots&&\cdots&&\vdots\\
0&0&\cdots&1&-1\\
\end{bmatrix}$$

# CVXY Solver

In [51]:
x = cp.Variable(m + 1)

In [52]:
prr.shape

(10000, 101)

In [54]:
obj = cp.Minimize(cp.sum(cp.square(prr@x - Mr)/n))

In [55]:
constraints = [F@x >= 0]

In [56]:
prob = cp.Problem(obj, constraints)
prob.solve(solver=cp.MOSEK, verbose=False)

0.001336957615084046

In [58]:
M = np.array(x.value)

In [61]:
M

array([ 0.36414112, -0.0016409 , -0.0016409 , -0.0016409 , -0.0016409 ,
       -0.0016409 , -0.0016409 , -0.0016409 , -0.0016409 , -0.0016409 ,
       -0.0016409 , -0.0016409 , -0.0016409 , -0.0016409 , -0.0016409 ,
       -0.0016409 , -0.0016409 , -0.0016409 , -0.0016409 , -0.0016409 ,
       -0.0016409 , -0.0016409 , -0.0016409 , -0.0016409 , -0.0016409 ,
       -0.0016409 , -0.0016409 , -0.0016409 , -0.0016409 , -0.0016409 ,
       -0.0016409 , -0.0016409 , -0.0016409 , -0.0016409 , -0.0016409 ,
       -0.0016409 , -0.0016409 , -0.0016409 , -0.0016409 , -0.00164091,
       -0.00164091, -0.00164091, -0.00164091, -0.00164091, -0.00164091,
       -0.00164091, -0.00164091, -0.00164091, -0.00164091, -0.00164091,
       -0.00164091, -0.00164091, -0.00164091, -0.00164091, -0.00164091,
       -0.00164091, -0.00164091, -0.00164091, -0.00164091, -0.00164091,
       -0.00164092, -0.00164092, -0.00164092, -0.00164092, -0.00164092,
       -0.00164092, -0.00164092, -0.00164092, -0.00164092, -0.00

In [59]:
# Considering there are 1000 users with uniform sampled rank in [1, m+1]
np.random.seed(0)
idx = np.random.randint(0, m, 1000)

In [60]:
M[idx].mean()

0.0049431464465987795